## AIHub Json Parsing

### Development Environment

In [ ]:
%pip install kss==3.7.3

KSS Argument Error: Restart Jupyter Kernel Runtime

In [ ]:
%pip install python-mecab-ko

KSS 3.7.3 matches python-mecab-ko

In [ ]:
%pip install pandas

In [1]:
import re
import os
import kss
import ray
import json
from mecab import MeCab
import pandas as pd
from glob import glob
from itertools import chain

In [2]:
pwd

'D:\\AIHUB'

### Function

In [3]:
def sorted_list(path_list):
    path_list = sorted(path_list, reverse=False)
    path_list = sorted(path_list, key=len)
    
    return path_list

In [ ]:
def json_file_name_list(path_list):
    
    file_name  = [glob(i, recursive = True) for i in path_list][0]
    file_name = sorted_list(file_name)
    
    return file_name

In [4]:
def train_valid_json_file_name_list(path_list):

  train_file_name, valid_file_name = [glob(i, recursive = True) if 'rain' in i
                                      else glob(i, recursive = True)
                                      for i in path_list]

  train_file_name = sorted_list(train_file_name)
  valid_file_name = sorted_list(valid_file_name)
    
  return train_file_name, valid_file_name

In [5]:
def divide_source_file_list(l, n): 
    
  for i in range(0, len(l), n): 
    yield l[i:i + n] 

In [6]:
def txt_file_name_list(source_file_nested_list, folder_corpus_type_name):

  text_file_name_list = [folder_corpus_type_name + str(i) + ".txt"
                              for i in range(len(source_file_nested_list))]
    
  return text_file_name_list

In [ ]:
def post_txt_file_name_list(corpus_list):
   
  post_corpus_list = [corpus_file.replace("pro", "post")
                      for corpus_file in corpus_list]

  return post_corpus_list

In [ ]:
def formal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거


    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, text)    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, text)    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', source[:2])) == True:
        source = re.sub(source[:2], "", source)
        
    source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)
    # '가.', '나.', ... 형태의 문자열 제거  
        
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False
    

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, + 제외
            # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
            # 다섯 어절 이하 문장 제외


            if ']' in sentence and '[' not in sentence:
                sentence  = re.sub(r".*?]", "", sentence)    
            # 중괄호 앞에 있는 '성명/직함]' 형태 제거


            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거

            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외

                for sentence2 in kss.split_sentences(sentence, use_heuristic=False,
                                        num_workers=32):
                    for sentence3 in kss.split_sentences(sentence2, use_heuristic=False,
                                                         num_workers=32):
                        preprocessing_sentence_list.append(sentence3)

            # 마지막에 KSS(Korean Sentence Segmentation)로 문장 분리 2번 실행

  
    return preprocessing_sentence_list

In [76]:
def tableqa_preprocessing_text(source):
    
    preprocessing_sentence_list = []
    source_split = re.split('br|tr|td|table|tbody|rowspan', source)
    # 문자열 'br', 'tr', 'td', 'table', 'tbody'를 기준으로 문장 분리

    for source in source_split:
        sentence = re.sub('(br|tr|td|table|tbody|rowspan|<|>|/)', '', source)
        # 문자열 'br', 'tr', 'td', 'table', 'tbody' 특수기호 '<', '>', '/'를 제외
            
        sentence = sentence.strip()
        # strip으로 앞뒤 공백 제거
        
        sentence = re.sub(r"\[.*?\]|\{.*?\}", "", sentence)
        # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거

        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(sentence)

    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(sentence)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = sentence.find(text) + len(text)
            if len(sentence) >= last_index+1 and sentence[last_index-text_size-1] == '(' and sentence[last_index+1] == '.':
                sentence = sentence.replace(sentence[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                sentence = sentence.replace(small_bracket, text)    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = sentence.find(text) + len(text)
                if len(sentence) >= last_index+1 and sentence[last_index-text_size-1] == '(' and sentence[last_index+1] == '.':
                    sentence = sentence.replace(sentence[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    sentence = sentence.replace(small_bracket, text)    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', sentence[:2])) == True:
        sentence = re.sub(sentence[:2], "", sentence)
        
    sentence = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", sentence)
    # '가.', '나.', ... 형태의 문자열 제거 
        

        if len(sentence) > 1 and \
        re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
        bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and  \
        len(sentence.split()) > 5:
         # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, +, 
        # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
        # 다섯 어절 이하 문장 제외
        
            
            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거
            
            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                for sentence2 in kss.split_sentences(sentence, use_heuristic=False,
                                        num_workers=32):
                    for sentence3 in kss.split_sentences(sentence2, use_heuristic=False,
                                                         num_workers=32):
                        preprocessing_sentence_list.append(sentence3)

            # 마지막에 KSS(Korean Sentence Segmentation)로 문장 분리 2번 실행


    return preprocessing_sentence_list

### AIHUB 행정 문서 대상 기계독해 데이터

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=569)

In [10]:
path_list = ['AIHUB_행정 문서 대상 기계독해 데이터/Training/'+ '/**/*.json', 
             'AIHUB_행정 문서 대상 기계독해 데이터/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [11]:
the_number_of_file = len(train_file_name) + len(valid_file_name)
print("The number of file:", the_number_of_file)
n = 2
print("The number of list element:", n)
train_file_nested_list = list(divide_source_file_list(train_file_name, n))
valid_file_nested_list = list(divide_source_file_list(valid_file_name, n))

The number of file: 12
The number of list element: 2


In [12]:
train_text_file_name_list = txt_file_name_list(train_file_nested_list,
                                                           "exploration/machine_reading_data_for_administrative_documents_pro/AIHUB_machine_reading_data_for_administrative_documents_train_")
valid_text_file_name_list = txt_file_name_list(valid_file_nested_list,
                                                           "exploration/machine_reading_data_for_administrative_documents_pro/AIHUB_machine_reading_data_for_administrative_documents_valid_")

In [13]:
def list_length_checker(source_file_nested_list, batch_size):
    
    the_number_of_total_txt_file = 0
    the_number_of_txt_file_list = []
    temp_nested_index = []
    
    for source_file_list in source_file_nested_list:   
        temp_index = []
        for i in range(len(source_file_list)):

            if type(source_file_list) == str:
                source_file = source_file_list

            elif type(source_file_list) == list:
                source_file = source_file_list[i]

            with open(source_file, 'r', encoding='utf-8') as one_json_file:
                one_json_sample = json.load(one_json_file) 

            source_df = pd.DataFrame(one_json_sample['data'])
            source_dict = dict(source_df['paragraphs'].explode())
            source_json = pd.json_normalize(source_dict)
            source_list = list(source_json.filter(regex='context').values[0])
            
            the_number_of_txt_file = ((len(source_list) // batch_size) + 1)

            if len(source_list) >= 1000:
                print("File:", source_file)    
                print("Index:", i, "  ", "Length of Source List:", len(source_list), \
                    "  ", "The number of txt file:", the_number_of_txt_file, "\n")
                the_number_of_txt_file_list.append(the_number_of_txt_file)
                the_number_of_total_txt_file  += the_number_of_txt_file
            else:
                the_number_of_total_txt_file  += 1
                the_number_of_txt_file_list.append(1)
                if i > 0 and the_number_of_file % i == 0:
                    temp_index.append(i)
                    try:
                        if i == temp_nested_index[0][0] and len(temp_nested_index) <= 1:
                            print("[For Example]")
                            print("This is not subject of batch. It's small source list.")                            
                            print("File:", source_file)
                            print("Length of Source List:", len(source_list), 
                                    "  ", "The number of txt file:", 1, "\n") 
                    except:
                        pass

    print("Batch Size:", batch_size)
    print("The number of txt file:", the_number_of_total_txt_file)
    
    return the_number_of_total_txt_file, the_number_of_txt_file_list

In [24]:
def make_corpus_txt_with_batch_list(source_file_nested_list,
                                    text_file_name_list,
                                    batch_size, the_number_of_total_txt_file_list):

  print("[Size]")
  print("The number of preprocessing corpus: " + str(sum(the_number_of_total_txt_file_list)))
  print("\n[Order]")
  num = 0
  for i in range(len(source_file_nested_list)):
    source_file_list = source_file_nested_list[i]

    for j in range(len(source_file_list)):

      if type(source_file_list) == str:
        source_file = source_file_list

      elif type(source_file_list) == list:
        source_file = source_file_list[j]

      with open(source_file, 'r', encoding='utf-8') as one_json_file:
        one_json_sample = json.load(one_json_file)
        
      source_df = pd.DataFrame(one_json_sample['data'])
      source_dict = dict(source_df['paragraphs'].explode())
      source_json = pd.json_normalize(source_dict)
      source_list = list(source_json.filter(regex='context').values[0])
      
      n = batch_size
      source_batch_list = list(divide_source_file_list(source_list, n))
      
      for source_list in source_batch_list:
          num += 1
          print(str(num), end=" ")  
          
          if 'tableqa' in source_file:

              with open(os.path.join('AIHUB_corpus/' + text_file_name_list[i][:-4] + "_" + str(num) + "_tableqa.txt"), "a", encoding='utf-8') as fp:        
                  fp.write("\n".join(source_list))   
    
          else:

              with open(os.path.join('AIHUB_corpus/' + text_file_name_list[i][:-4] + "_" + str(num) + ".txt"), "a", encoding='utf-8') as fp:        
                  fp.write("\n".join(source_list))   

In [31]:
batch_size = 10000
the_number_of_train_txt_file, the_number_of_train_txt_file_list = list_length_checker(train_file_nested_list, batch_size)

File: AIHUB_행정 문서 대상 기계독해 데이터/Training\TS_multiple_choice.json
Index: 0    Length of Source List: 30170    The number of txt file: 4 

File: AIHUB_행정 문서 대상 기계독해 데이터/Training\TS_span_extraction.json
Index: 1    Length of Source List: 127864    The number of txt file: 13 

File: AIHUB_행정 문서 대상 기계독해 데이터/Training\TS_span_extraction_how.json
Index: 0    Length of Source List: 58148    The number of txt file: 6 

File: AIHUB_행정 문서 대상 기계독해 데이터/Training\TS_tableqa.json
Index: 1    Length of Source List: 125814    The number of txt file: 13 

File: AIHUB_행정 문서 대상 기계독해 데이터/Training\TS_text_entailment.json
Index: 0    Length of Source List: 33744    The number of txt file: 4 

File: AIHUB_행정 문서 대상 기계독해 데이터/Training\TS_unanswerable.json
Index: 1    Length of Source List: 19656    The number of txt file: 2 

Batch Size: 10000
The number of txt file: 42


In [ ]:
batch_size = 10000
make_corpus_txt_with_batch_list(train_file_nested_list, train_text_file_name_list,
                batch_size, the_number_of_train_txt_file_list)

In [32]:
batch_size = 10000
the_number_of_valid_txt_file, the_number_of_valid_txt_file_list = list_length_checker(valid_file_nested_list, batch_size)

File: AIHUB_행정 문서 대상 기계독해 데이터/Validation\VS_multiple_choice.json
Index: 0    Length of Source List: 5626    The number of txt file: 1 

File: AIHUB_행정 문서 대상 기계독해 데이터/Validation\VS_span_extraction.json
Index: 1    Length of Source List: 23970    The number of txt file: 3 

File: AIHUB_행정 문서 대상 기계독해 데이터/Validation\VS_span_extraction_how.json
Index: 0    Length of Source List: 10916    The number of txt file: 2 

File: AIHUB_행정 문서 대상 기계독해 데이터/Validation\VS_tableqa.json
Index: 1    Length of Source List: 23578    The number of txt file: 3 

File: AIHUB_행정 문서 대상 기계독해 데이터/Validation\VS_text_entailment.json
Index: 0    Length of Source List: 6308    The number of txt file: 1 

File: AIHUB_행정 문서 대상 기계독해 데이터/Validation\VS_unanswerable.json
Index: 1    Length of Source List: 3686    The number of txt file: 1 

Batch Size: 10000
The number of txt file: 11


In [ ]:
batch_size = 10000
make_corpus_txt_with_batch_list(valid_file_nested_list, valid_text_file_name_list,
                batch_size, the_number_of_valid_txt_file_list)

In [ ]:
pro_total_corpus_list = glob("AIHUB_corpus/exploration/machine_reading_data_for_administrative_documents_pro/AIHUB_machine_reading_data_for_administrative_documents_train_" +"*.txt")

In [ ]:
len(pro_total_corpus_list)

In [ ]:
post_total_corpus_list = post_txt_file_name_list(pro_total_corpus_list)

In [ ]:
line_list = []
line_num = 0
with open(pro_total_corpus_list[0], 'r', encoding='utf-8') as f:
    lines = f.read().splitlines() 
    for line in lines:
        line_num += 1
        if line_num <= 1:
           line_list.append(line)
for line in line_list:
    print(line, end="\n\n")

In [ ]:
line_list = []
line_num = 0
with open(pro_total_corpus_list[0], 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()
    for line in lines:
        line_num += 1
        if line_num <= 1:  
            sentences = formal_preprocessing_text(line)
            for sentence in sentences:
                line_list.append(sentence) 
            
for line in line_list:
    print(line, end="\n\n")

In [ ]:
ray.init(num_cpus = 4)

@ray.remote

def formal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거


    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, text)    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, text)    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', source[:2])) == True:
        source = re.sub(source[:2], "", source)
        
    source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)
    # '가.', '나.', ... 형태의 문자열 제거 
        
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False
    

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, + 제외
            # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
            # 다섯 어절 이하 문장 제외


            if ']' in sentence and '[' not in sentence:
                sentence  = re.sub(r".*?]", "", sentence)    
            # 중괄호 앞에 있는 '성명/직함]' 형태 제거


            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거

            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외

                for sentence2 in kss.split_sentences(sentence, use_heuristic=False,
                                        num_workers=32):
                    for sentence3 in kss.split_sentences(sentence2, use_heuristic=False,
                                                         num_workers=32):
                        preprocessing_sentence_list.append(sentence3)

            # 마지막에 KSS(Korean Sentence Segmentation)로 문장 분리 2번 실행

  
    return preprocessing_sentence_list

def tableqa_preprocessing_text(source):
    
    preprocessing_sentence_list = []
    source_split = re.split('br|tr|td|table|tbody|rowspan', source)
    # 문자열 'br', 'tr', 'td', 'table', 'tbody'를 기준으로 문장 분리

    for source in source_split:
        sentence = re.sub('(br|tr|td|table|tbody|rowspan|<|>|/)', '', source)
        # 문자열 'br', 'tr', 'td', 'table', 'tbody' 특수기호 '<', '>', '/'를 제외
            
        sentence = sentence.strip()
        # strip으로 앞뒤 공백 제거
        
        sentence = re.sub(r"\[.*?\]|\{.*?\}", "", sentence)
        # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거

        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(sentence)

    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(sentence)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = sentence.find(text) + len(text)
            if len(sentence) >= last_index+1 and sentence[last_index-text_size-1] == '(' and sentence[last_index+1] == '.':
                sentence = sentence.replace(sentence[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                sentence = sentence.replace(small_bracket, text)    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = sentence.find(text) + len(text)
                if len(sentence) >= last_index+1 and sentence[last_index-text_size-1] == '(' and sentence[last_index+1] == '.':
                    sentence = sentence.replace(sentence[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    sentence = sentence.replace(small_bracket, text)    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', sentence[:2])) == True:
        sentence = re.sub(sentence[:2], "", sentence)
        
    sentence = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", sentence)
    # '가.', '나.', ... 형태의 문자열 제거 
        

        if len(sentence) > 1 and \
        re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
        bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and  \
        len(sentence.split()) > 5:
         # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, +, 
        # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
        # 다섯 어절 이하 문장 제외
        
            
            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거
            
            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                for sentence2 in kss.split_sentences(sentence, use_heuristic=False,
                                        num_workers=32):
                    for sentence3 in kss.split_sentences(sentence2, use_heuristic=False,
                                                         num_workers=32):
                        preprocessing_sentence_list.append(sentence3)

            # 마지막에 KSS(Korean Sentence Segmentation)로 문장 분리 2번 실행


    return preprocessing_sentence_list

In [ ]:
print("[Size]")
print("The number of preprocessing corpus: " + str(len(pro_total_corpus_list)))
print("\n[Order]")
num = 0
process_num = 10    

for pro, post in zip(pro_total_corpus_list, post_total_corpus_list):
    
    sentence_list = []
    
    with open(pro, 'r', encoding='utf-8') as f:
        lines = f.read().splitlines() 
        nested_lines_num = len(lines) // process_num
        for i in range(nested_lines_num - 1):
            start_line = process_num * i
            end_line = process_num * (i+1)
            
            if 'tableqa' in pro:
                futures = [tableqa_preprocessing_text.remote(lines[start_line:end_line][j]) for j in range(process_num)]
                results = ray.get(futures)

                if i == nested_lines_num - 2:
                    futures = [tableqa_preprocessing_text.remote(lines[end_line:][j]) for j in range(len(lines) - end_line)]
                    results = ray.get(futures)
                    
            else:
                futures = [formal_preprocessing_text.remote(lines[start_line:end_line][j]) for j in range(process_num)]
                results = ray.get(futures)
                if i == nested_lines_num - 2:
                    futures = [formal_preprocessing_text.remote(lines[end_line:][j]) for j in range(len(lines) - end_line)]
                    results = ray.get(futures)
                
            sentences = list(chain.from_iterable(results))
            sentence_list.append(sentences)
    
    sentence_list = list(chain.from_iterable(sentence_list))
    
    num += 1
    print(str(num), end=" ")  
        
    with open(post, 'a', encoding='utf-8') as fp:
        fp.write("\n".join(sentence_list))

In [ ]:
ray.shutdown()

In [ ]:
corpus_list = glob("AIHUB_corpus/exploration/machine_reading_data_for_administrative_documents_post/AIHUB_machine_reading_data_for_administrative_documents_" +"*.txt")

In [ ]:
with open('AIHUB_corpus/duplicate/AIHUB_machine_reading_data_for_administrative_documents.txt', 'w') as f:
    for corpus in corpus_list:
        with open(corpus) as text:
            for line in text:
                f.write(line)

In [ ]:
with open('AIHUB_corpus/AIHUB_machine_reading_data_for_administrative_documents.txt', 'w', encoding='utf-8') as f1:
    with open('AIHUB_corpus/duplicate/AIHUB_machine_reading_data_for_administrative_documents.txt', encoding='utf-8') as f2:
        lines = f2.read().splitlines()
        single_sentence_dict = dict.fromkeys(lines)
        single_sentence_list = list(single_sentence_dict)
        f1.write("\n".join(single_sentence_list))